In [1]:
from google.colab import files
files.upload()   # choose kaggle.json from your computer


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"halimaakhter","key":"6e7e4c254836ab2c243217292b7e81f9"}'}

In [2]:
# 3. move it into the right place
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 4. test
!kaggle competitions list | head -n 10

ref                                                                              deadline             category                reward  teamCount  userHasEntered  
-------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/arc-prize-2025                               2025-11-03 23:59:00  Featured         1,000,000 Usd        916           False  
https://www.kaggle.com/competitions/make-data-count-finding-data-references      2025-09-09 23:59:00  Research           100,000 Usd       1197           False  
https://www.kaggle.com/competitions/jigsaw-agile-community-rules                 2025-10-24 06:59:00  Featured           100,000 Usd       1100           False  
https://www.kaggle.com/competitions/mitsui-commodity-prediction-challenge        2025-10-06 23:59:00  Featured           100,000 Usd        775           False  
https://www.kaggle.com/compe

In [3]:
!kaggle competitions download -c llm-classification-finetuning -p ./llm_pref
!unzip -q ./llm_pref/llm-classification-finetuning.zip -d ./llm_pref


  0% 0.00/57.0M [00:00<?, ?B/s]
100% 57.0M/57.0M [00:00<00:00, 667MB/s]


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss
import numpy as np

# 1.1 load
train = pd.read_csv("llm_pref/train.csv")
test  = pd.read_csv("llm_pref/test.csv")
sub   = pd.read_csv("llm_pref/sample_submission.csv")  # keeps the exact column names



# Peek columns; typical ones you’ll see:
# id, prompt, response_a, response_b, winner (e.g., 'A'/'B' or 1/0)
print(train.head(2))




      id             model_a     model_b  \
0  30192  gpt-4-1106-preview  gpt-4-0613   
1  53567           koala-13b  gpt-4-0613   

                                              prompt  \
0  ["Is it morally right to try to have a certain...   
1  ["What is the difference between marriage lice...   

                                          response_a  \
0  ["The question of whether it is morally right ...   
1  ["A marriage license is a legal document that ...   

                                          response_b  winner_model_a  \
0  ["As an AI, I don't have personal beliefs or o...               1   
1  ["A marriage license and a marriage certificat...               0   

   winner_model_b  winner_tie  
0               0           0  
1               1           0  


In [8]:
# --- Make labels: y = 1 if A preferred, 0 if B preferred
# The dataset has winner_model_a / winner_model_b / winner_tie
y_all = train["winner_model_a"].astype(int)

# Simple, robust choice: DROP ties from training.
train_nt = train[train["winner_tie"] == 0].reset_index(drop=True)
y = y_all.loc[train_nt.index].reset_index(drop=True)

In [9]:
# Build text inputs (prompt + both answers)
def join_row(df):
    return (df["prompt"].fillna("") + " [SEP] A: " +
            df["response_a"].fillna("") + " [SEP] B: " +
            df["response_b"].fillna(""))

X_text_tr  = join_row(train_nt)
X_text_te  = join_row(test)

1. Simple Baseline — TF-IDF + Logistic Regression (fast CPU)

1.   List item
2.   List item



In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss

X_tr, X_val, y_tr,y_val = train_test_split(X_text_tr,y,test_size=0.2,stratify=y,random_state=42)

pipe = Pipeline( [
    ("tfidf", TfidfVectorizer(ngram_range=(1,2),min_df = 3,max_df=0.95,
                              strip_accents="unicode",sublinear_tf=True)),
    ("clf",LogisticRegression(solver="liblinear",C=2.0,max_iter=2000,class_weight="balanced"))


])

pipe.fit(X_tr,y_tr)

val_p = pipe.predict_proba(X_val)[:,1]



# Train on all non-tie data
pipe.fit(X_text_tr, y)
test_p = pipe.predict_proba(X_text_te)[:,1]

# Build submission (prob that A is preferred)
out = sub.copy()
pred_col = [c for c in out.columns if c.lower() not in ["id","row_id"]][0]  # auto-detect the pred column
out[pred_col] = test_p
out.to_csv("submission_tfidf_lr.csv", index=False)






In [11]:
from google.colab import files
files.download("submission_tfidf_lr.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>